In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Read the CSV and Perform Basic Data Cleaning

In [5]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
pd.set_option('display.max_rows', 86)
df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
initial_list_status            object
out_prncp                     float64
out_prncp_inv                 float64
total_pymnt                   float64
total_pymnt_inv               float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_a

In [10]:
#df['hardship_flag'].value_counts()
df = pd.get_dummies(df, columns=['hardship_flag', 'debt_settlement_flag', 'application_type', 'pymnt_plan', 'initial_list_status', 'home_ownership', ])
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,debt_settlement_flag,hardship_flag_N
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,1
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,1
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,1
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,1
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,1


In [8]:
#df['debt_settlement_flag'].value_counts()
df = pd.get_dummies(df, columns=['debt_settlement_flag', 'application_type', 'pymnt_plan', 'initial_list_status', 'issue_d', 'next_pymnt_d', 'verification_status', 'home_ownership'])
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,0,1,0,1,0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,1,0,0,1,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,1,0,0,1,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,0,1,0,0,1,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,1,1,0,0,0,1,0,0


In [9]:
df.dtypes

loan_amnt                              float64
int_rate                               float64
installment                            float64
annual_inc                             float64
loan_status                             object
dti                                    float64
delinq_2yrs                            float64
inq_last_6mths                         float64
open_acc                               float64
pub_rec                                float64
revol_bal                              float64
total_acc                              float64
out_prncp                              float64
out_prncp_inv                          float64
total_pymnt                            float64
total_pymnt_inv                        float64
total_rec_prncp                        float64
total_rec_int                          float64
total_rec_late_fee                     float64
recoveries                             float64
collection_recovery_fee                float64
last_pymnt_am

# Split the Data into Training and Testing

In [16]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)
X.head()


,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_OWN,home_ownership_RENT,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,1,0,0,1,0,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0,0,0,0,1,0,1,0,0,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0,0,0,0,1,0,1,0,0,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,1,0,0,1,0,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0,0,0,0,1,0,1,1,0,0


In [17]:
# Create our target
y = df["loan_status"]
y[:5]

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [18]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_OWN,home_ownership_RENT,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.106747,0.357659,0.371696,0.451066,0.177238,0.383161,0.616839,0.478007,0.373992,0.148001
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.308793,0.479314,0.483261,0.497603,0.381873,0.486161,0.486161,0.499520,0.483865,0.355104
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [20]:
scaler = StandardScaler()

In [22]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)

In [23]:
X_scaler = scaler.fit(X_train)

In [24]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [25]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [26]:
# Calculated the balanced accuracy score
predictions = brf.predict(X_test_scaled)

In [27]:
acc_score = accuracy_score(y_test, predictions)

In [28]:
acc_score

0.8945655332752107

In [29]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [30]:
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,60,27
Actual 1,1787,15331


Accuracy Score : 0.8945655332752107
Classification Report
              precision    recall  f1-score   support

   high_risk       0.03      0.69      0.06        87
    low_risk       1.00      0.90      0.94     17118

    accuracy                           0.89     17205
   macro avg       0.52      0.79      0.50     17205
weighted avg       0.99      0.89      0.94     17205



In [71]:
#y_pred_rf = brf.predict(X_test)
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.69      0.90      0.06      0.79      0.60        87
   low_risk       1.00      0.90      0.69      0.94      0.79      0.63     17118

avg / total       0.99      0.89      0.69      0.94      0.79      0.63     17205



In [31]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
importances

array([1.02585718e-02, 2.66845488e-02, 1.54548894e-02, 1.52676924e-02,
       1.78508915e-02, 3.80262910e-03, 5.44752010e-03, 9.70404922e-03,
       2.13471110e-03, 1.56815582e-02, 1.11501517e-02, 1.64226086e-02,
       1.50942372e-02, 5.02504003e-02, 5.74754027e-02, 8.16286743e-02,
       5.49516442e-02, 6.27230374e-03, 0.00000000e+00, 0.00000000e+00,
       6.47092259e-02, 2.18560519e-04, 0.00000000e+00, 0.00000000e+00,
       3.67367476e-03, 1.02598300e-02, 5.68493904e-03, 8.48482611e-03,
       3.87516318e-03, 8.17533740e-03, 1.41898545e-02, 1.09175683e-02,
       1.19832812e-02, 5.08981726e-03, 6.89061617e-03, 1.51051741e-02,
       1.21328117e-02, 1.32767347e-02, 5.45010831e-03, 9.32676676e-03,
       1.07753301e-02, 9.06725483e-03, 1.45090087e-02, 1.50032564e-02,
       1.74345464e-02, 1.88340677e-04, 4.57983856e-05, 1.39936950e-02,
       1.52162117e-02, 9.68370511e-03, 7.25207265e-03, 7.85783468e-03,
       1.15659789e-02, 1.51288283e-02, 3.69522109e-03, 1.06745099e-02,
      

In [32]:
importances_df = pd.DataFrame(sorted(zip(brf.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True) 
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)

In [33]:
importances_df

,Feature Importances
1,
total_rec_prncp,0.081629
last_pymnt_amnt,0.064709
total_pymnt_inv,0.057475
total_rec_int,0.054952
total_pymnt,0.050250
int_rate,0.026685
issue_d_Jan-2019,0.019272
dti,0.017851
bc_util,0.017435


### Easy Ensemble AdaBoost Classifier

In [34]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [35]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
y_train.ravel

# GBC - Learning Rate Determination and Performance Metrics

In [36]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=learning_rate,
        random_state=0)
    model.fit(X_train_scaled,y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training)".format(
        model.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation)".format(
        model.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training)
Accuracy score (validation)

Learning rate:  0.1
Accuracy score (training)
Accuracy score (validation)

Learning rate:  0.25
Accuracy score (training)
Accuracy score (validation)

Learning rate:  0.5
Accuracy score (training)
Accuracy score (validation)

Learning rate:  0.75
Accuracy score (training)
Accuracy score (validation)

Learning rate:  1
Accuracy score (training)
Accuracy score (validation)



In [101]:
gbc = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=.05,
    random_state=0)



In [102]:
gbc.fit(X_train,y_train.ravel())

# Score the model
print("Accuracy score (training)".format(
   gbc.score(
        X_train,
        y_train)))
print("Accuracy score (validation)".format(
    model.score(
        X_test,
        y_test)))

Accuracy score (training): 0.998
Accuracy score (validation): 0.994


In [103]:
gbc_pred = gbc.predict(X_test)

# Generate accuracy score for predictions using y_test
accuracy_score(y_test, gbc_pred)

0.9961057832025574

In [111]:
gbc_cm = confusion_matrix(y_test, gbc_pred)
gbc_cm_df = pd.DataFrame(
    gbc_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [112]:
display(gbc_cm_df)

,Predicted 0,Predicted 1
Actual 0,31,56
Actual 1,11,17107


# AdaBoost Performance Summary

In [104]:
# Train the Classifier
abc = AdaBoostClassifier(n_estimators=100, random_state=78)

In [94]:
model = abc.fit(X_train, y_train)

In [81]:
abc_predictions = model.predict(X_test)

In [82]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, abc_predictions)

In [83]:
acc_score

0.9965126416739319

In [106]:
# Display the confusion matrix
abc_cm = confusion_matrix(y_test, abc_predictions)
cm_df = pd.DataFrame(
    abc_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [107]:
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,33,54
Actual 1,6,17112


In [109]:
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, abc_predictions))

Accuracy Score : 0.9965126416739319
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.38      1.00      0.52      0.62      0.36        87
   low_risk       1.00      1.00      0.38      1.00      0.62      0.40     17118

avg / total       1.00      1.00      0.38      1.00      0.62      0.40     17205

